In [28]:
import os
import torch
import os
import math
import torch
import torchdyn
import numpy as np
import matplotlib.pyplot as plt

from typing import List
from dataclasses import dataclass,asdict,field


from markov_bridges.data.abstract_dataloader import (
    MarkovBridgeDataloader,
    MarkovBridgeDataset,
    MarkovBridgeDataClass
)

from markov_bridges.data.music_dataloaders import (
    get_data
)

from markov_bridges.configs.config_classes.data.basics_configs import MarkovBridgeDataConfig

from torch.distributions import Categorical
from markov_bridges.configs.config_classes.data.music_configs import LakhPianoRollConfig

In [21]:
def generate_noise_sample(dataset,data_config:MarkovBridgeDataConfig):
    dataset_size = dataset.size(0)
    generation_dimension = data_config.dimensions - data_config.context_dimension
    if data_config.source_discrete == "uniform":
        vocab_size = data_config.vocab_size
        NoiseDistribution = Categorical(torch.full((vocab_size,),1./vocab_size))
        noise_sample = NoiseDistribution.sample((dataset_size,generation_dimension))
        return noise_sample
    else:
        raise Exception("Source not Implemented")

In [2]:
data_config = LakhPianoRollConfig(context_discrete=True)
train_data,  test_data, descramble_key = get_data(data_config)

In [23]:
test_source = generate_noise_sample(test_data,data_config)

In [25]:
train_source.shape

torch.Size([6000, 224])

In [29]:
dataclass = MarkovBridgeDataClass()

In [31]:
dataset = train_data

def obtain_data_divisions(dataset,data_config:MarkovBridgeDataConfig)->MarkovBridgeDataClass:
    """
    """
    # source
    source_discrete = generate_noise_sample(train_data,data_config)

    # context
    if data_config.context_dimension > 0:
        context_discrete = dataset[:,:data_config.context_dimension]
    else:
        context_discrete = None

    # target
    target_discrete = dataset[:,data_config.context_dimension:]
    dataclass.target_continuous = None

    return MarkovBridgeDataClass(context_discrete=context_discrete,
                                 )

In [ ]:
"""
num_samples = 100
tensor_size = 10

dataset = MarkovBridgeDataset(num_samples, tensor_size)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)        
"""

In [32]:
#databatch = next(dataloader.__iter__())

In [33]:
# databatch.target_node

# Defining Dataconfigs

In [35]:
@dataclass
class  AbstractDataloaderConfig:
    dimensions: int = 4
    vocab_size: int = 2

## Music

In [36]:
from markov_bridges import data_path
from typing import List

from dataclasses import asdict,field

from markov_bridges.data.music_dataloaders import (
    get_data
)

@dataclass
class LakhPianoRollConfig(MarkovBridgeDataConfig):
    name:str = "LakhPianoRoll"
    dataset_name:str = "lakh_roll" # emnist, fashion, mnist

    batch_size: int= 32
    data_dir:str = str(data_path / "raw")

    dimensions: int = 256
    vocab_size: int = 129
    conditional_dimension:int = 32
    generation_dimension:int = 224

    total_data_size: int = 6973
    training_size: int = 6000
    test_size: int = 973
    test_split: float = None

    temporal_net_expected_shape : List[int] = None
    data_min_max: List[float] = field(default_factory=lambda:[0.,128.])

    def __post_init__(self):
        self.dimensions, self.temporal_net_expected_shape = self.dimensions, [self.dimensions]
        self.number_of_labels = None
        self.test_split = self.test_size/float(self.total_data_size)
        self.generation_dimension = self.dimensions - self.conditional_dimension

In [37]:
data_config = LakhPianoRollConfig()
data_config

LakhPianoRollConfig(context_discrete=False, context_continuous=False, target_discrete=True, target_continuous=False, source_discrete='uniform', source_continuous=None, name='LakhPianoRoll', dataset_name='lakh_roll', dimensions=256, vocab_size=129, max_training_size=None, max_test_size=None, batch_size=32, data_dir='C:\\Users\\cesar\\Desktop\\Projects\\DiffusiveGenerativeModelling\\OurCodes\\markov_bridges\\data\\raw', conditional_dimension=32, generation_dimension=224, total_data_size=6973, training_size=6000, test_size=973, test_split=0.13953821884411302, temporal_net_expected_shape=[256], data_min_max=[0.0, 128.0])

In [38]:
train_data,  test_data, descramble_key = get_data(data_config)

In [40]:
test_data.shape

torch.Size([973, 256])

## Discrete

In [1]:


@dataclass
class  AbstractDataloaderConfig:
    dimensions: int = 4
    vocab_size: int = 2
    


@dataclass
class CategoricalDataloaderConfig(AbstractDataloaderConfig):
    name:str = "CategoricalDataloader"
    dataset_name:str = "categorical_dirichlet" # categorical_dirichlet
    data_dir:str = image_data_path

    dirichlet_alpha:float = None
    categorical_probability:float = None

    dimensions: int = 4
    vocab_size: int = 2

    batch_size: int = 23
    test_split: float = 0.2
    total_data_size: int = 60000
    
    temporal_net_expected_shape : List[int] = None
    data_min_max: List[float] = field(default_factory=lambda:[0.,1.])

    def __post_init__(self):
        self.temporal_net_expected_shape =  [self.dimensions]



NameError: name 'dataclass' is not defined

# MOONS

In [ ]:
from torchdyn.datasets import generate_moons
from utils import sample_8gaussians,sample_moons

In [7]:
generate_moons(10, noise=0.2)

(tensor([[ 1.0469,  0.0469],
         [ 0.8699,  0.8699],
         [ 0.0194,  1.0194],
         [-0.6167,  0.7975],
         [-0.9594,  0.0406],
         [ 0.0766,  0.5766],
         [ 0.3884, -0.1116],
         [ 1.0736, -0.4264],
         [ 1.8040, -0.1102],
         [ 2.1741,  0.6741]]),
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1]))

In [5]:
batch_size = 32
x0 = sample_8gaussians(batch_size)
x1 = sample_moons(batch_size)